In [1]:
import numpy as np
from IO import IO
from PC_denoiser import PC_denoiser
from PointFilter.Customer_Module.chamfer_distance.dist_chamfer import chamferDist
import torch
import os
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


c:\Users\azhenis4\AppData\Local\anaconda3\envs\fyp_env\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from tqdm import tqdm
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [3]:
cloud_pairs = [
    ("Tetrahedron.npy", "Tetrahedron_0.005.npy"),
    ("fandisk_100K.npy", "fandisk_100K_0.005.npy"), 
    ("cube_uniform_100K.npy", "cube_uniform_100K_0.005.npy"),
    ("boxunion2_100K.npy", "boxunion2_100K_0.005.npy")
]

In [4]:
results = {}
for gt_path, noisy_path in cloud_pairs:
    results[gt_path] = {}
    gt = IO.get_arr_from(os.path.join("data", gt_path))
    noisy = IO.get_arr_from(os.path.join("data", noisy_path))
    for name in dir(PC_denoiser):
        # Get the attribute
        if name.startswith("denoise"):
            method = getattr(PC_denoiser, name)

            start_time = time.time()
            denoised = method(noisy)
            execution_time = time.time() - start_time

            loss = PC_denoiser.calculate_chamfer_distance_loss(gt, denoised)

            results[gt_path][name] = {"time": execution_time, "loss": loss}

            path = os.path.join("images", f"{gt_path[:-4]}_{name}.png")
            IO.save_visualization(cloud=denoised, save_path=path)
results

Denoise Clusters:   0%|          | 0/4 [00:00<?, ?it/s]

Denoise Clusters:   0%|          | 0/4 [00:00<?, ?it/s]

Denoise Clusters:   0%|          | 0/4 [00:00<?, ?it/s]

Denoise Clusters:   0%|          | 0/4 [00:00<?, ?it/s]

{'Tetrahedron.npy': {'denoise_dmr': {'time': 17.290677547454834,
   'loss': 1.4142421605356503e-05},
  'denoise_mls': {'time': 0.9985833168029785, 'loss': 2.0020983356516808e-05},
  'denoise_pointcleannet': {'time': 114.04175543785095,
   'loss': 1.3582616702478845e-05},
  'denoise_pointfilter': {'time': 125.81558585166931,
   'loss': 7.705009920755401e-06},
  'denoise_score_based': {'time': 37.84608197212219,
   'loss': 9.14813790586777e-06},
  'denoise_voxel_grid': {'time': 0.30309319496154785,
   'loss': 3.7261892430251464e-05}},
 'fandisk_100K.npy': {'denoise_dmr': {'time': 19.4921658039093,
   'loss': 1.3872664567315951e-05},
  'denoise_mls': {'time': 1.068164348602295, 'loss': 2.5100067432504147e-05},
  'denoise_pointcleannet': {'time': 114.50656914710999,
   'loss': 1.2520291420514695e-05},
  'denoise_pointfilter': {'time': 128.14853692054749,
   'loss': 7.302269295905717e-06},
  'denoise_score_based': {'time': 38.202563762664795,
   'loss': 9.421052709512878e-06},
  'denoise_vo

In [7]:
for gt_path, noisy_path in cloud_pairs:
    results[gt_path] = {}
    gt = IO.get_arr_from(os.path.join("data", gt_path))
    noisy = IO.get_arr_from(os.path.join("data", noisy_path))
    path = os.path.join("images", f"{gt_path[:-4]}_ground_truth.png")
    IO.save_visualization(cloud=gt, save_path=path)

    path = os.path.join("images", f"{gt_path[:-4]}_noisy.png")
    IO.save_visualization(cloud=noisy, save_path=path)

# Example

In [14]:
path = os.path.join("data", "Tetrahedron_0.005.npy")
pcl = IO.get_arr_from(path)
pcl

array([[-0.14818473,  0.01119605, -0.16039269],
       [-0.23029459,  0.19200414, -0.13042861],
       [ 0.18336986,  0.12867783,  0.03038543],
       ...,
       [-0.03029756,  0.09184797, -0.22520615],
       [-0.08969136,  0.22849035, -0.1309408 ],
       [-0.26883158,  0.01869855, -0.05352968]], dtype=float32)

In [15]:
IO.visualize_arr(pcl)

In [18]:
denoised = PC_denoiser.denoise_score_based(pcl)

Denoise Clusters:   0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
IO.visualize_arr(denoised)